# dotnet/try Github Dashboard <img src ="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/207px-Jupyter_logo.svg.png" width="60px" alt="dotnet bot in space" align ="right">

### Add NuGet package references 

In [1]:
#r "nuget:Octokit, 0.32.0"
#r "nuget:NodaTime, 2.4.6"
using Octokit;
using NodaTime;
using NodaTime.Extensions;
using XPlot.Plotly;

Installing package Octokit, version 0.32.0................done!

Successfully added reference to package Octokit, version 0.32.0

Installing package NodaTime, version 2.4.6....done!

Successfully added reference to package NodaTime, version 2.4.6

### Setup
Create a GitHub public API client 

In [2]:
var organization = "dotnet";
var repositoryName = "try";
var options = new ApiOptions();
var gitHubClient = new GitHubClient(new ProductHeaderValue("notebook"));


[Generate a user token](https://help.github.com/en/github/authenticating-to-github/creating-a-personal-access-token-for-the-command-line) to get rid of public [api](https://github.com/octokit/octokit.net/blob/master/docs/getting-started.md) throttling policies for anonymous users 

In [1]:
//var tokenAuth = new Credentials("YOUR TOKEN");
//gitHubClient.Credentials = tokenAuth;

Create checkpoints for the:  current day(today), the start of the current month and, the start of the current year. 

In [3]:
var today = SystemClock.Instance.InUtc().GetCurrentDate();
var startOfTheMonth = today.With(DateAdjusters.StartOfMonth);
var startOfTheYear = new LocalDate(today.Year, 1, 1).AtMidnight();

Query GitHub for : 
- Issues created this month
- Issues closed this month
- Every issue this year

In [4]:
var createdIssuesRequest = new RepositoryIssueRequest
            {
                Since = startOfTheMonth.ToDateTimeUnspecified(),
                Filter = IssueFilter.Created
            };
var closedIssuesRequest = new RepositoryIssueRequest
            {
                Since = startOfTheMonth.ToDateTimeUnspecified(),
                State = ItemStateFilter.Closed
            };
var thisYearIssuesRequest = new RepositoryIssueRequest
            {
                State = ItemStateFilter.All,
                Since = startOfTheYear.ToDateTimeUnspecified()
            };

Start pulling data via the GitHub API

In [5]:
var createdThisMonth = await gitHubClient.Issue.GetAllForRepository(organization, repositoryName, createdIssuesRequest);
var closedThisMonth = await gitHubClient.Issue.GetAllForRepository(organization, repositoryName, closedIssuesRequest);
var thisYearIssues = await gitHubClient.Issue.GetAllForRepository(organization, repositoryName, thisYearIssuesRequest);

Group open and closed issues by month 

In [6]:
var openSoFar = createdThisMonth.OrderBy(i => i.CreatedAt).Where(i => i.State.StringValue == "open").ToArray();
var openByMonthOfCreation = openSoFar.GroupBy(i => new { i.CreatedAt.Year, i.CreatedAt.Month})
                .Select(g => new {Date = g.Key, Count = g.Count()}).ToArray();

var closedSoFar = thisYearIssues.OrderBy(i => i.CreatedAt).Where(i => i.State.StringValue == "closed").ToArray();
var closedByMonthOfClosure = closedSoFar.GroupBy(i => new { i.ClosedAt.Value.Year, i.ClosedAt.Value.Month})
                .Select(g => new {Date = g.Key, Count = g.Count()}).ToArray();
var totalOpenIssues = thisYearIssues.Count();
var openCountByMonth = closedSoFar.GroupBy(i => new { i.CreatedAt.Year, i.CreatedAt.Month})
                .Select(g => {
                    var count = g.Count();                    
                    var dataPoint = new  {Date = g.Key, Count = totalOpenIssues};
                    totalOpenIssues -= count;
                    return dataPoint;
                }).ToArray();

Show issues opened this month grouped by day 

In [7]:
var createdThisMonthByDay = createdThisMonth.GroupBy(i => new DateTime(i.CreatedAt.Year,i.CreatedAt.Month, i.CreatedAt.Day)); 
var lineChart = Chart.Line(createdThisMonthByDay.Select(g => new Tuple<DateTime,int>(g.Key, g.Count())));
lineChart.WithTitle("Daily Issues");
display(lineChart);

In [8]:
display(openSoFar.Select(i => new {i.CreatedAt, i.Title, State = i.State.StringValue,  i.Number}).OrderByDescending(d => d.CreatedAt));

index,CreatedAt,Title,State,Number
0,2019-10-28 23:11:11Z,properly format non-generic `IDictionary`,open,571
1,2019-10-28 20:52:54Z,Custom formatter for F# discriminated unions,open,570
2,2019-10-28 19:17:38Z,Support spaces after `%%` in magic commands,open,569
3,2019-10-28 13:36:47Z,How do I set the platform to x64 for the C# kernel in Jupyter notebook?,open,568
4,2019-10-27 17:02:53Z,Graph error when running Tensorflow Transfer Learning on .NET Kernel,open,565
5,2019-10-27 15:52:32Z,SqlClientFactory.Instance is null when using Microsoft.Data.SqlClient in Jupyter C# Kernel,open,564
6,2019-10-27 15:50:18Z,Unable to use System.Data.SqlClient in Jupyter C# Kernel,open,563
7,2019-10-25 22:38:59Z,Support Bloggers and Code Samples,open,561
8,2019-10-25 15:08:40Z,F# plotting with xplot gives an error in the first scatterplot,open,559
9,2019-10-25 00:24:50Z,Implicitly ignore the result of `display`,open,558


In [9]:
var lineChart = Chart.Line(openByMonthOfCreation.Select(g => new Tuple<DateTime, int>(new DateTime(g.Date.Year, g.Date.Month, 1),g.Count)));
lineChart.WithTitle("Issues still opened grouped by month");
display(lineChart);

In [10]:
var idleByMonth = openSoFar.Where( i => i.Comments == 0).GroupBy(i => new DateTime( i.CreatedAt.Year, i.CreatedAt.Month, 1))
    .Select(g => new {Date = g.Key, Count = g.Count()}).ToArray();
var activeByMonth = openSoFar.Where( i => i.Comments > 0).GroupBy(i => new DateTime( i.CreatedAt.Year, i.CreatedAt.Month, 1))
                .Select(g => new {Date = g.Key, Count = g.Count()}).ToArray();

var idleSeries = new Graph.Scattergl
{
    name = "Idle",
    y = idleByMonth.Select(g => g.Count ).ToArray(),
    x = idleByMonth.Select(g => g.Date ).ToArray()
};

var activeSeries = new Graph.Scattergl
{
    name = "Active",
    y = activeByMonth.Select(g => g.Count ).ToArray(),
    x = activeByMonth.Select(g => g.Date ).ToArray()
};


var chart = Chart.Plot(new[] {idleSeries, activeSeries});
chart.WithTitle("Idle and active open issue report");
display(chart);

In [11]:
var openDataPoints = openByMonthOfCreation
    .Select(g => new { y = g.Count, x = new DateTime(g.Date.Year, g.Date.Month, 1)} )
    .OrderBy(d => d.x).ToArray();


var closedDataPoints = closedByMonthOfClosure
    .Select(g => new { y = g.Count, x = new DateTime(g.Date.Year, g.Date.Month, 1)} )
    .OrderBy(d => d.x).ToArray();

var openCountByMonthDataPoints = openCountByMonth
    .Select(g => new { y = g.Count, x = new DateTime(g.Date.Year, g.Date.Month, 1)} )
    .OrderBy(d => d.x).ToArray();

var openSeries = new Graph.Scattergl
{
    name = "Created",
    y = openDataPoints.Select(g => g.y ).ToArray(),
    x = openDataPoints.Select(g => g.x ).ToArray()
};

var closeSeries = new Graph.Scattergl
{
    name = "Closed",
    y = closedDataPoints.Select(g => g.y ).ToArray(),
    x = closedDataPoints.Select(g => g.x ).ToArray()
};

var stillOpenSeries = new Graph.Scattergl
{
    name = "Open",
    y = openCountByMonthDataPoints.Select(g => g.y ).ToArray(),
    x = openCountByMonthDataPoints.Select(g => g.x ).ToArray()
};


var chart = Chart.Plot(new[] {openSeries, closeSeries, stillOpenSeries});
chart.WithTitle("Issue report for the current year");
display(chart);

### How many times has dotnet/try been forked ?

In [12]:
var forks = await gitHubClient.Repository.Forks.GetAll(organization, repositoryName);

In [13]:
var forkCreateByMonth = forks.GroupBy(f => new DateTime(f.CreatedAt.Year, f.CreatedAt.Month,  f.CreatedAt.Day) )
    .Select(g => new {Date = g.Key, Count = g.Count()}).OrderBy(g => g.Date).ToArray();
var forkLastUpdateByMonth = forks.GroupBy(f => new DateTime(f.UpdatedAt.Year, f.UpdatedAt.Month,  f.UpdatedAt.Day) )
    .Select(g => new {Date = g.Key, Count = g.Count()}).OrderBy(g => g.Date).ToArray();

var total = 0;
var forkCountByMonth = forkCreateByMonth.OrderBy(g => g.Date).Select(g => new {g.Date, Count = total += g.Count}).ToArray();

var forkCreationSeries = new Graph.Scattergl
{
    name = "created by month",
    y = forkCreateByMonth.Select(g => g.Count ).ToArray(),
    x = forkCreateByMonth.Select(g => g.Date ).ToArray()
};

var forkTotalSeries = new Graph.Scattergl
{
    name = "running total",
    y = forkCountByMonth.Select(g => g.Count ).ToArray(),
    x = forkCountByMonth.Select(g => g.Date ).ToArray()
};

var forkUpdateSeries = new Graph.Scattergl
{
    name = "last update by month",
    y = forkLastUpdateByMonth.Select(g => g.Count ).ToArray(),
    x = forkLastUpdateByMonth.Select(g => g.Date ).ToArray()
};



var chart = Chart.Plot(new[] {forkCreationSeries,forkTotalSeries,forkUpdateSeries});
chart.WithTitle("Fork activity");
display(chart);